In [3]:
class SVM_Loss(torch.nn.modules.Module):
    """
    SVM Loss function
    """    
    def __init__(self):
        """
        Initialize the SVM Loss function
        """
        super(SVM_Loss,self).__init__()

    def forward(self, outputs, labels):
        """
        Forward pass of the SVM Loss function
        """
        return torch.sum(torch.clamp(1 - outputs.t()*labels, min=0))/batch_size

In [4]:
# torch parameters being used
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
logging.info(f'Torch Version: {TORCH_VERSION}, Cuda Version: {CUDA_VERSION}')

INFO:root:Torch Version: 1.10, Cuda Version: cu113


INFO:root:Resnet Config File Read Successfully.


In [6]:
# create datasets and dataloaders
train_dataset, val_dataset, test_dataset, class_names, num_classes = create_datasets(config["data_dir"], config["train_perc"], config["val_perc"], config["test_perc"])
logging.info('Train, Validation and Test Datasets Created Successfully.')

dataloaders, dataset_sizes = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config["batch_size"], num_workers = config["num_workers"])
logging.info('Train, Validation and Test Dataloaders Created Successfully.')

INFO:root:Train, Validation and Test Datasets Created Successfully.
INFO:root:Train, Validation and Test Dataloaders Created Successfully.


In [7]:
# Parameters for training
#batch_size = 8
#num_epochs = 35 
input_size = 150528  #(dimension of image 224*224*3)
#num_classes = 3
learning_rate = 0.0001 ## step size used by SGD 
momentum = 0.1 ## Momentum is a moving average of our gradients (helps to keep direction)

In [8]:
# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
logging.info(f'Device {device} Being Used.')

INFO:root:Device cuda:0 Being Used.


In [9]:
# SVM regression model and Loss
svm_model = nn.Linear(input_size,num_classes)

# Loss and optimizer
svm_loss_criteria = SVM_Loss()
svm_optimizer = torch.optim.SGD(svm_model.parameters(), lr=learning_rate, momentum=momentum)
total_step = len(dataloaders["train"])

In [10]:
svm_model = svm_model.to(device) 

In [11]:
num_epochs = 1

In [12]:
batch_size = config["batch_size"]

In [13]:
for epoch in range(num_epochs):
    avg_loss_epoch = 0
    batch_loss = 0
    total_batches = 0
    for images, labels in dataloaders["train"]:
        images = images.to(device)
        labels = labels.to(device)
        images = images.reshape(-1, input_size)
        svm_optimizer.zero_grad()
        
        # Forward pass        
        outputs = svm_model(images)           
        loss_svm = svm_loss_criteria(outputs, labels)    
        
        # Backward and optimize
        loss_svm.backward()
        svm_optimizer.step()    
        total_batches += 1     
        batch_loss += loss_svm.item()

    # Print loss every few iterations
    avg_loss_epoch = batch_loss/total_batches
    print ('Epoch [{}/{}], Averge Loss:for epoch {}: {:.4f}]'.format(epoch+1, num_epochs, epoch+1, avg_loss_epoch))

/hpc/group/rescomp/sg623/miniconda3/envs/cv_540/lib/python3.8/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/1], Averge Loss:for epoch 1: 2.6598]


In [ ]:
correct = 0.
total = 0.
for images, labels in dataloaders["val"]:

    # Reshape images
    images = images.reshape(-1, input_size)
    
    # Forward pass
    outputs = svm_model(images) 
    
    # Get predictions
    predicted = torch.argmax(outputs, axis=1)

    # Calculate accuracy
    total += labels.size(0) 
    correct += (predicted == labels).sum()    

print('Accuracy of the SVM model on the val images: %f %%' % (100 * (correct.float() / total)))

In [ ]:
# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
logging.info(f'Device {device} Being Used.')

# train the model
model_dir = config["model_dir"]
num_epochs = config["num_epochs"]

logging.info('Started Training The Model.\n')

net = train_model(model = net, model_dir = model_dir, criterion = criterion, optimizer = optimizer, dataloaders = dataloaders, dataset_sizes = dataset_sizes, scheduler = lr_scheduler, device = device, num_epochs = num_epochs)